In [25]:
import pandas as pd
import numpy as np

In [26]:
PATH = '.\crawl_json\crawl_0_0.json'

df = pd.read_json(PATH, orient='index')
df.drop(['authors', 'image_url', 'localpath', 'title_page', 'title_rss'], axis = 1, inplace = True)
df.head(2)

,date_download,date_modify,date_publish,description,language,source_domain,maintext,title,url
0,"06/09/2021, 10:14:38","06/09/2021, 10:14:38","03/28/2020, 14:58:03",Neighbors » Forums » General Discussion » Arti...,en,neighbors.denverpost.com,Neighbors » Forums » General Discussion » Arti...,"Denver Post poll: Most Coloradans support NRA,...",http://neighbors.denverpost.com/viewtopic.php?...
1,"06/09/2021, 10:14:38","06/09/2021, 10:14:38","03/28/2020, 16:15:00",Neighbors » Forums » Sports News » Denver Bron...,en,neighbors.denverpost.com,Neighbors » Forums » Sports News » Denver Bron...,Elvis Dumervil agrees to contract with Baltimo...,http://neighbors.denverpost.com/viewtopic.php?...


In [29]:
import nltk

ModuleNotFoundError: No module named 'nlk'

In [31]:
df['tokenized_100_words'] = df['maintext'].apply(nltk.word_tokenize)
df.head(1)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\Nicolas/nltk_data'
    - 'C:\\Users\\Nicolas\\anaconda3\\envs\\kcc\\nltk_data'
    - 'C:\\Users\\Nicolas\\anaconda3\\envs\\kcc\\share\\nltk_data'
    - 'C:\\Users\\Nicolas\\anaconda3\\envs\\kcc\\lib\\nltk_data'
    - 'C:\\Users\\Nicolas\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************
